In [ ]:



import numpy as np

def dct_matrix(n):
    D = np.zeros((n, n))
    for k in range(n):
        for i in range(n):
            if k == 0:
                D[k, i] = np.sqrt(1/n)
            else:
                D[k, i] = np.sqrt(2/n) * np.cos(np.pi*(2*i+1)*k/(2*n))
    return D

def to_q8_8(value):
    # Convert float to Q8.8 fixed point format.
    return int(round(value * 256)) & 0xFFFF


n = 8
D = dct_matrix(n)
# print(D)
q_matrix = [[to_q8_8(D[i,j]) for j in range(n)] for i in range(n)]
i=0
for row in q_matrix:
    for x in row:
        print("bram_D[{}] <= 16'h{:04X};".format(i,x),end=' ')
        i+=1


bram_D[0] <= 16'h005B; bram_D[1] <= 16'h005B; bram_D[2] <= 16'h005B; bram_D[3] <= 16'h005B; bram_D[4] <= 16'h005B; bram_D[5] <= 16'h005B; bram_D[6] <= 16'h005B; bram_D[7] <= 16'h005B; bram_D[8] <= 16'h007E; bram_D[9] <= 16'h006A; bram_D[10] <= 16'h0047; bram_D[11] <= 16'h0019; bram_D[12] <= 16'hFFE7; bram_D[13] <= 16'hFFB9; bram_D[14] <= 16'hFF96; bram_D[15] <= 16'hFF82; bram_D[16] <= 16'h0076; bram_D[17] <= 16'h0031; bram_D[18] <= 16'hFFCF; bram_D[19] <= 16'hFF8A; bram_D[20] <= 16'hFF8A; bram_D[21] <= 16'hFFCF; bram_D[22] <= 16'h0031; bram_D[23] <= 16'h0076; bram_D[24] <= 16'h006A; bram_D[25] <= 16'hFFE7; bram_D[26] <= 16'hFF82; bram_D[27] <= 16'hFFB9; bram_D[28] <= 16'h0047; bram_D[29] <= 16'h007E; bram_D[30] <= 16'h0019; bram_D[31] <= 16'hFF96; bram_D[32] <= 16'h005B; bram_D[33] <= 16'hFFA5; bram_D[34] <= 16'hFFA5; bram_D[35] <= 16'h005B; bram_D[36] <= 16'h005B; bram_D[37] <= 16'hFFA5; bram_D[38] <= 16'hFFA5; bram_D[39] <= 16'h005B; bram_D[40] <= 16'h0047; bram_D[41] <= 16'hFF82; br

In [ ]:
test = np.ones((8,8))
sol = D@test@D.T
ans = [[to_q8_8(sol[i,j]) for j in range(n)] for i in range(n)]
for row in ans:
        print(" ".join("{:04X}".format(x) for x in row))

0800 0000 0000 0000 0000 0000 0000 0000
0000 0000 0000 0000 0000 0000 0000 0000
0000 0000 0000 0000 0000 0000 0000 0000
0000 0000 0000 0000 0000 0000 0000 0000
0000 0000 0000 0000 0000 0000 0000 0000
0000 0000 0000 0000 0000 0000 0000 0000
0000 0000 0000 0000 0000 0000 0000 0000
0000 0000 0000 0000 0000 0000 0000 0000


In [57]:
def q8_8_to_float(value: int) -> float:
    # Interpret the 16-bit value as signed
    if value & 0x8000:  # If sign bit is set
        value -= 0x10000  # Two's complement adjustment
    return value / 256.0

# Example input
hex_input = 0x3200
result = q8_8_to_float(hex_input)
result


50.0

In [37]:
import cv2
test = cv2.imread('img.png',cv2.IMREAD_GRAYSCALE)-128
sol = D@test@D.T
ans = [[to_q8_8(sol[i,j]) for j in range(n)] for i in range(n)]
for row in ans:
    print(" ".join("{:04X}".format(x) for x in row))


FA40 FFCB FE8F FFB0 FFC0 0005 FFC9 0251
0059 006D FF75 FF9D 0012 FFAC 008B 006D
FF36 0017 006D 0042 0023 FF9E 002D FF8C
004B FFDD FFF2 0013 FFCE FFED FFF2 0094
FFC0 005D FFDD FFBC 0040 FFF2 0054 FF75
0032 FFEC FFBC 0013 004B 0013 FFBC FFDD
000E FFF6 002D FFE5 FFAC 0029 0013 0030
0012 FF93 005D 0054 FFA7 001D FFA3 006D


In [4]:
import numpy as np
from scipy.fftpack import dct, idct

def dct2(matrix):
    """Compute the 2D DCT (type-II) with orthonormal normalization."""
    return dct(dct(matrix, axis=0, norm='ortho'), axis=1, norm='ortho')

def idct2(matrix):
    """Compute the 2D inverse DCT (type-II) with orthonormal normalization."""
    return idct(idct(matrix, axis=0, norm='ortho'), axis=1, norm='ortho')

def to_q8_8(val):
    """
    Convert a floating-point value to Q8.8 fixed-point format,
    represented as a signed 16-bit integer.
    """
    return np.int16(np.round(val * 256))

def q8_8_format(matrix):
    """
    Convert a 2D numpy array of floating-point numbers to a string
    of Q8.8 fixed-point hexadecimal values.
    """
    q8_8_matrix = np.empty_like(matrix, dtype=np.int16)
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            q8_8_matrix[i, j] = to_q8_8(matrix[i, j])
    # Build a multi-line hex string.
    lines = []
    for row in q8_8_matrix:
        # Cast each element to a Python int to avoid overflow issues
        line = " ".join("{:04X}".format(int(x) & 0xFFFF) for x in row)
        lines.append(line)
    return "\n".join(lines)

def parse_q8_8_str(q8_8_str):
    """
    Parse a multi-line string of Q8.8 hexadecimal values into an 8x8 float matrix.
    """
    tokens = q8_8_str.split()
    coeffs = []
    for token in tokens:
        val = int(token, 16)
        # Convert from unsigned to signed (16-bit two's complement)
        if val & 0x8000:
            val -= 0x10000
        coeffs.append(val)
    coeffs = np.array(coeffs, dtype=np.int16).reshape((8, 8))
    # Convert from Q8.8 fixed-point to float.
    return coeffs.astype(np.float32) / 256.0

def compute_rmse_psnr(original, reconstructed):
    """
    Compute RMSE and PSNR between the original and reconstructed matrices.
    """
    mse = np.mean((original - reconstructed) ** 2)
    rmse = np.sqrt(mse)
    if mse == 0:
        psnr = float('inf')
    else:
        psnr = 20 * np.log10(255.0 / rmse)
    return rmse, psnr

def generate_valid_matrix():
    """
    Generate a random 8x8 matrix with values 0 to 255 whose forward DCT 
    coefficients all lie within the range [-127, 127]. If not, try again.
    """
    while True:
        M = np.random.randint(-64, 64, (8, 8)).astype(np.float32)
        dct_coeffs = dct2(M)
        if np.all(dct_coeffs >= -127) and np.all(dct_coeffs <= 127):
            return M, dct_coeffs
        
import os
import cv2

sample_gray_image = os.path.join('..', 'images', 'lena_gray_256.tif')
sample_gray_image = cv2.imread(sample_gray_image, cv2.IMREAD_UNCHANGED)

sample_gray_image_blocks = np.zeros((sample_gray_image.shape[0] // 8, sample_gray_image.shape[1] // 8, 8, 8), dtype=sample_gray_image.dtype)
for i in range(0, sample_gray_image.shape[0], 8):
    for j in range(0, sample_gray_image.shape[1], 8):
        sample_gray_image_blocks[i // 8, j // 8] = sample_gray_image[i:i + 8, j:j + 8]

def main():
    np.random.seed(2308)  # For reproducibility
    
    # Step 1: Generate a valid random 8x8 matrix.
    # original_matrix, dct_coeffs = generate_valid_matrix()
    original_matrix = sample_gray_image_blocks[0, 0]
    dct_coeffs = dct2(original_matrix)
    print("Original 8x8 Matrix (values 0-255):")
    print(original_matrix)
    print()
    cnt = 0
    for i in range(8):
        for j in range(8):
            print(f"dut.bram_X[{cnt}] = 16'h{int(to_q8_8(original_matrix[i][j]))& 0xFFFF:04X};",end=' ')
            cnt+=1
    print()
    print()
    
    # Step 2: Compute its forward 2D DCT (already done in generate_valid_matrix) 
    # and print the coefficients (float format).
    print("DCT Coefficients (float) in range [-127, 127]:")
    print(np.round(dct_coeffs, 2))
    print()
    
    # Step 3: Convert the DCT coefficients to Q8.8 fixed-point hexadecimal format.
    q8_8_str = q8_8_format(dct_coeffs)
    print("DCT Coefficients in Q8.8 fixed-point format (hex):")
    print(q8_8_str)
    print()
    
    # Step 4: Simulate user input by using the Q8.8 string and parse it.
    user_input = """e1d8 060f fa43 00b0 03e6 faa9 ff63 00dc 
0efc 03ce fb23 00f0 024f fb37 050e fe58 
0183 0181 00bd ff8b 0320 00f0 0072 fd9f 
fe6b fa96 fef5 01e0 fe6b 001c 00f1 0234 
fd0b 023c 0139 fe1c feb3 0060 fcf0 00bb 
007b 027c ffe7 0065 0070 ffce 0138 fe71 
ff7c fe87 023b ff71 0021 0069 ffc5 0000 
02f7 fe23 fbf1 0175 009f ff49 0102 0110""" # In practice, this string could be provided by the user.
    parsed_dct = parse_q8_8_str(user_input)
    
    # Reconstruct the matrix using the inverse 2D DCT.
    reconstructed_matrix = idct2(parsed_dct)
    print("Reconstructed 8x8 Matrix:")
    print(np.round(reconstructed_matrix, 2))
    print()
    
    # Step 5: Compute RMSE and PSNR between the original and reconstructed matrices.
    rmse, psnr = compute_rmse_psnr(original_matrix, reconstructed_matrix)
    print("RMSE: {:.4f}".format(rmse))
    print("PSNR: {:.2f} dB".format(psnr))
    
main()


Original 8x8 Matrix (values 0-255):
[[162 162 162 163 165 162 155 160]
 [162 162 162 163 165 162 155 160]
 [162 162 162 163 165 162 155 160]
 [160 163 160 159 159 156 155 156]
 [155 158 159 157 163 158 159 156]
 [156 156 156 155 158 157 159 158]
 [158 157 157 159 160 158 156 156]
 [158 159 155 158 155 151 157 156]]

dut.bram_X[0] = 16'hA200; dut.bram_X[1] = 16'hA200; dut.bram_X[2] = 16'hA200; dut.bram_X[3] = 16'hA300; dut.bram_X[4] = 16'hA500; dut.bram_X[5] = 16'hA200; dut.bram_X[6] = 16'h9B00; dut.bram_X[7] = 16'hA000; dut.bram_X[8] = 16'hA200; dut.bram_X[9] = 16'hA200; dut.bram_X[10] = 16'hA200; dut.bram_X[11] = 16'hA300; dut.bram_X[12] = 16'hA500; dut.bram_X[13] = 16'hA200; dut.bram_X[14] = 16'h9B00; dut.bram_X[15] = 16'hA000; dut.bram_X[16] = 16'hA200; dut.bram_X[17] = 16'hA200; dut.bram_X[18] = 16'hA200; dut.bram_X[19] = 16'hA300; dut.bram_X[20] = 16'hA500; dut.bram_X[21] = 16'hA200; dut.bram_X[22] = 16'h9B00; dut.bram_X[23] = 16'hA000; dut.bram_X[24] = 16'hA000; dut.bram_X[25] = 

11.5

In [65]:

import numpy as np

def dct_matrix(n):
    D = np.zeros((n, n))
    for k in range(n):
        for i in range(n):
            if k == 0:
                D[k, i] = np.sqrt(1/n)
            else:
                D[k, i] = np.sqrt(2/n) * np.cos(np.pi*(2*i+1)*k/(2*n))
    return D

def to_q11_5(value):
    # Convert float to Q8.8 fixed point format.
    return int(round(value * 32)) & 0xFFFF


n = 8
D = dct_matrix(n)
# print(D)
q_matrix = [[to_q11_5(D[i,j]) for j in range(n)] for i in range(n)]
i=0
for row in q_matrix:
    for x in row:
        print("bram_D[{}] <= 16'h{:04X};".format(i,x),end=' ')
        i+=1


bram_D[0] <= 16'h000B; bram_D[1] <= 16'h000B; bram_D[2] <= 16'h000B; bram_D[3] <= 16'h000B; bram_D[4] <= 16'h000B; bram_D[5] <= 16'h000B; bram_D[6] <= 16'h000B; bram_D[7] <= 16'h000B; bram_D[8] <= 16'h0010; bram_D[9] <= 16'h000D; bram_D[10] <= 16'h0009; bram_D[11] <= 16'h0003; bram_D[12] <= 16'hFFFD; bram_D[13] <= 16'hFFF7; bram_D[14] <= 16'hFFF3; bram_D[15] <= 16'hFFF0; bram_D[16] <= 16'h000F; bram_D[17] <= 16'h0006; bram_D[18] <= 16'hFFFA; bram_D[19] <= 16'hFFF1; bram_D[20] <= 16'hFFF1; bram_D[21] <= 16'hFFFA; bram_D[22] <= 16'h0006; bram_D[23] <= 16'h000F; bram_D[24] <= 16'h000D; bram_D[25] <= 16'hFFFD; bram_D[26] <= 16'hFFF0; bram_D[27] <= 16'hFFF7; bram_D[28] <= 16'h0009; bram_D[29] <= 16'h0010; bram_D[30] <= 16'h0003; bram_D[31] <= 16'hFFF3; bram_D[32] <= 16'h000B; bram_D[33] <= 16'hFFF5; bram_D[34] <= 16'hFFF5; bram_D[35] <= 16'h000B; bram_D[36] <= 16'h000B; bram_D[37] <= 16'hFFF5; bram_D[38] <= 16'hFFF5; bram_D[39] <= 16'h000B; bram_D[40] <= 16'h0009; bram_D[41] <= 16'hFFF0; br

In [ ]:
import numpy as np
from scipy.fftpack import dct, idct

def dct2(matrix):
    """Compute the 2D DCT (type-II) with orthonormal normalization."""
    return dct(dct(matrix, axis=0, norm='ortho'), axis=1, norm='ortho')

def idct2(matrix):
    """Compute the 2D inverse DCT (type-II) with orthonormal normalization."""
    return idct(idct(matrix, axis=0, norm='ortho'), axis=1, norm='ortho')

def to_q11_5(val):
    """
    Convert a floating-point value to Q11.5 fixed-point format,
    represented as a signed 16-bit integer.
    """
    # Multiply by 2^5 = 32
    return np.int16(np.round(val * 32))

def q11_5_format(matrix):
    """
    Convert a 2D numpy array of floating-point numbers to a string
    of Q11.5 fixed-point hexadecimal values.
    """
    q11_5_matrix = np.empty_like(matrix, dtype=np.int16)
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            q11_5_matrix[i, j] = to_q11_5(matrix[i, j])
    # Build a multi-line hex string.
    lines = []
    for row in q11_5_matrix:
        # Cast each element to Python int to avoid overflow issues
        line = " ".join("{:04X}".format(int(x) & 0xFFFF) for x in row)
        lines.append(line)
    return "\n".join(lines)

def parse_q11_5_str(q11_5_str):
    """
    Parse a multi-line string of Q11.5 hexadecimal values into an 8x8 float matrix.
    """
    tokens = q11_5_str.split()
    coeffs = []
    for token in tokens:
        val = int(token, 16)
        # Convert from unsigned to signed (16-bit two's complement)
        if val & 0x8000:
            val -= 0x10000
        coeffs.append(val)
    coeffs = np.array(coeffs, dtype=np.int16).reshape((8, 8))
    # Convert from Q11.5 fixed-point to float (divide by 32)
    return coeffs.astype(np.float32) / 32.0

def compute_rmse_psnr(original, reconstructed):
    """
    Compute RMSE and PSNR between the original and reconstructed matrices.
    """
    mse = np.mean((original - reconstructed) ** 2)
    rmse = np.sqrt(mse)
    if mse == 0:
        psnr = float('inf')
    else:
        psnr = 20 * np.log10(255.0 / rmse)
    return rmse, psnr

def generate_valid_matrix():
    """
    Generate a random 8x8 matrix with values in the range [-64, 63] whose forward DCT 
    coefficients all lie within the range [-127, 127]. If not, try again.
    """
    while True:
        M = np.random.randint(-127,127, (8, 8)).astype(np.float32)
        dct_coeffs = dct2(M)
        return M, dct_coeffs

import os
import cv2

sample_gray_image = os.path.join('..', 'images', 'lena_gray_256.tif')
sample_gray_image = cv2.imread(sample_gray_image, cv2.IMREAD_UNCHANGED)

sample_gray_image_blocks = np.zeros((sample_gray_image.shape[0] // 8, sample_gray_image.shape[1] // 8, 8, 8), dtype=sample_gray_image.dtype)
for i in range(0, sample_gray_image.shape[0], 8):
    for j in range(0, sample_gray_image.shape[1], 8):
        sample_gray_image_blocks[i // 8, j // 8] = sample_gray_image[i:i + 8, j:j + 8]

def main():
    np.random.seed(123)  # For reproducibility
    
    # Step 1: Generate a valid random 8x8 matrix.
    # original_matrix, dct_coeffs = generate_valid_matrix()
    original_matrix = sample_gray_image_blocks[20, 15] - 128
    dct_coeffs = dct2(original_matrix)
    print("Original 8x8 Matrix (values in range [-64, 63]):")
    print(original_matrix)
    print()
    
    # Print each element of the original matrix in Q11.5 format
    cnt = 0
    for i in range(8):
        for j in range(8):
            print(f"dut.bram_X[{cnt}] = 16'h{int(to_q11_5(original_matrix[i][j])) & 0xFFFF:04X};", end=' ')
            cnt += 1
    print("\n")
    
    # Step 2: Print the DCT coefficients (float format) which are in range [-127, 127].
    print("DCT Coefficients (float) in range [-127, 127]:")
    print(np.round(dct_coeffs, 2))
    print()
    
    # Step 3: Convert the DCT coefficients to Q11.5 fixed-point hexadecimal format.
    q11_5_str = q11_5_format(dct_coeffs)
    print("DCT Coefficients in Q11.5 fixed-point format (hex):")
    print(q11_5_str)
    print()
    
    # Step 4: Simulate user input by using a provided Q11.5 hex string and parse it.
    user_input = """0f1c 007e 04b7 03e7 03f1 02c2 022a 013d 
fe61 fa9b fc2e fc18 fc70 fd73 fe00 ffbf 
fdb8 fd7b fcfc fd69 fd82 fee4 ff68 0066 
0502 07ee 07f4 064c 0549 051b 02fd 0196 
fc78 fac7 fb16 fb49 fc69 fcf9 fe50 ff41 
fec1 fe92 fe7a fedb ff71 ff95 ff2b ffa7 
04ff 06ec 06e4 05d8 04d2 03db 0313 018b 
fbfb f8cf f9e9 fafb fbb1 fc9f fdf3 feb2"""
    # In practice, the user would supply the above string.
    parsed_dct = parse_q11_5_str(user_input)
    parsed_dct_topk = parsed_dct.copy()
    parsed_dct_topk[3:,3:] = 0
    
    # Reconstruct the matrix using the inverse 2D DCT.
    reconstructed_matrix = idct2(parsed_dct_topk)
    print("Reconstructed 8x8 Matrix:")
    print(np.round(reconstructed_matrix, 2))
    print()
    
    # Step 5: Compute RMSE and PSNR between the original and reconstructed matrices.
    rmse, psnr = compute_rmse_psnr(original_matrix, reconstructed_matrix)
    print("RMSE: {:.4f}".format(rmse))
    print("PSNR: {:.2f} dB".format(psnr))
    
if __name__ == "__main__":
    main()


Original 8x8 Matrix (values in range [-64, 63]):
[[  5   3  16   9  14  14  24  22]
 [  4  10  12  10  16  16  21  20]
 [  0   7   8  12  19  21  20  25]
 [  5  10   6  14  16  13  15  21]
 [  5   6   7  12  13  18  17  24]
 [255   6   6   9  10  13  14  17]
 [  6   2   5  14  12  13  14  14]
 [  4   4  13   9  14  14  10  15]]

dut.bram_X[0] = 16'h00A0; dut.bram_X[1] = 16'h0060; dut.bram_X[2] = 16'h0200; dut.bram_X[3] = 16'h0120; dut.bram_X[4] = 16'h01C0; dut.bram_X[5] = 16'h01C0; dut.bram_X[6] = 16'h0300; dut.bram_X[7] = 16'h02C0; dut.bram_X[8] = 16'h0080; dut.bram_X[9] = 16'h0140; dut.bram_X[10] = 16'h0180; dut.bram_X[11] = 16'h0140; dut.bram_X[12] = 16'h0200; dut.bram_X[13] = 16'h0200; dut.bram_X[14] = 16'h02A0; dut.bram_X[15] = 16'h0280; dut.bram_X[16] = 16'h0000; dut.bram_X[17] = 16'h00E0; dut.bram_X[18] = 16'h0100; dut.bram_X[19] = 16'h0180; dut.bram_X[20] = 16'h0260; dut.bram_X[21] = 16'h02A0; dut.bram_X[22] = 16'h0280; dut.bram_X[23] = 16'h0320; dut.bram_X[24] = 16'h00A0; dut.